<a href="https://colab.research.google.com/github/0BVer/sch_AI/blob/main/sentiment_predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 예측

In [24]:
cd /content/drive/MyDrive/Colab Notebooks/data

/content/drive/MyDrive/Colab Notebooks/data


In [25]:
from keras.models import load_model
model = load_model('trained_model(epoch=6batch_size=64).h5')

In [27]:
import pandas as pd

df = pd.read_csv("Reviews.csv")
df = df[df['Score'] != 3]
df['sentiment'] = df['Score'].apply(lambda rating : 'positive' if rating > 3 else 'negative')
review_df1 = df[['Text', 'sentiment']]
test_sets1=review_df1.Text.values

df = pd.read_csv("Tweets.csv")
review_df2 = df[['text', 'airline_sentiment']]
review_df2 = review_df2[review_df2['airline_sentiment'] != 'neutral']
review_df2.columns = ["Text", "sentiment"]
test_sets2=review_df2.Text.values

review_df = pd.concat([review_df1, review_df2], ignore_index = True)
test_sets=review_df.Text.values
sentiment_label = review_df.sentiment.factorize()
answers = review_df.sentiment.values
length=len(test_sets1)

In [28]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=15000)
tokenizer.fit_on_texts(test_sets)

encoded_docs = tokenizer.texts_to_sequences(test_sets)

In [29]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
padded_sequence = pad_sequences(encoded_docs, maxlen = 200)

def predict_sentiment(text):
  tw = tokenizer.texts_to_sequences([text])
  tw = pad_sequences(tw, maxlen=200)
  prediction = int(model.predict(tw).round().item())
  return sentiment_label[1][prediction]


In [30]:
test_sentence1 = "I enjoy my trip on this flight."

print(predict_sentiment(test_sentence1))

positive


# 분류

In [ ]:
import re
import nltk
from nltk.corpus import stopwords
from wordcloud import WordCloud

nltk.download('stopwords')
stopwords = set(stopwords.words('english'))
stopwords.update(["br", "href", "would"])

positives=dict()
negatives=dict()
pos_cnt=0
neg_cnt=0
cnt=0

meta_text=""
pos_meta_text=""
neg_meta_text=""

for i in range(length):
  text=re.sub('^@[a-zA-Z0-9]*','',test_sets1[i])
  words=text.split(' ')
  words=[word for word in words if word not in stopwords]
  text=' '.join(words)
  text=''.join(list(filter(lambda x : x not in "!@#$%^&*())-='",text)))
  text=str(text).lower()
  text=re.sub('\[.*?\]', '',text)
  text=re.sub('https?://\S+|www\.\S+','',text)
  text=re.sub('<.*?>+','',text)
  text=re.sub(r'[^\w\s]','',text)
  text=re.sub('\n','',text)
  text=re.sub('\w*\d\w*','',text)
  result=predict_sentiment(text)

  words=text.split(' ')
  words=list(filter(None, words))
  words=[word for word in words if word not in stopwords]

  text=" ".join(words)
  meta_text+=text+" "

  if result=='positive':
    pos_cnt+=1
    for word in words:
      if word in positives:
        positives[word]+=1
      else:
        positives[word]=1
    pos_meta_text+=text+" "

  else:
    neg_cnt+=1
    for word in words:
      if word in negatives:
        negatives[word]+=1
      else:
        negatives[word]=1
    neg_meta_text+=text+" "
  
  if pos_cnt%500==0 or neg_cnt%500==0:
    print(pos_cnt, neg_cnt)
  if result==answers[i]:
    cnt+=1
  else:
    print(i+1,result,answers[i],"%d/%d"%(cnt,i+1))

import matplotlib.pyplot as plt

wordcloud = WordCloud(stopwords=stopwords).generate(meta_text)
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.savefig('wordcloud_total.png', pad_inches=0.1)
plt.show() 

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
204651 29500
204652 29500
204653 29500
204654 29500
204655 29500
204656 29500
204657 29500
204658 29500
204659 29500
204660 29500
204661 29500
204662 29500
204663 29500
204664 29500
204665 29500
204666 29500
204667 29500
204668 29500
204669 29500
204670 29500
204671 29500
204672 29500
204673 29500
204674 29500
204675 29500
204676 29500
204677 29500
204678 29500
204679 29500
204680 29500
204681 29500
204682 29500
204683 29500
204684 29500
204685 29500
234204 positive negative 221096/234204
234214 negative positive 221105/234214
234216 positive negative 221106/234216
234233 positive negative 221122/234233
234239 positive negative 221127/234239
234240 positive negative 221127/234240
234241 positive negative 221127/234241
234360 positive negative 221245/234360
234394 positive negative 221278/234394
205000 29521
234537 positive negative 221420/234537
234553 positive negative 221435/234553
234568 negative positive 221449/234568
234569 negative positive 221

In [7]:
import operator

positives=dict(positives)
negatives=dict(negatives)

sPositives=sorted(positives.items(), key=operator.itemgetter(1),reverse=True)
sNegatives=sorted(negatives.items(), key=operator.itemgetter(1),reverse=True)

sPositives=dict(sPositives)
sNegatives=dict(sNegatives)

pos_keys=list(sPositives.keys())
pos_values=list(sPositives.values())
neg_keys=list(sNegatives.keys())
neg_values=list(sNegatives.values())

top_pos_keys=pos_keys[:80]
top_pos_cnt=pos_values[:80]
top_neg_keys=neg_keys[:80]
top_neg_cnt=neg_values[:80]

cnt=0
i=0
while i < len(top_pos_keys):
  if top_pos_keys[i] in top_neg_keys:
    j=top_neg_keys.index(top_pos_keys[i])
    del top_pos_keys[i]
    del top_pos_cnt[i]
    del top_neg_keys[i]
    del top_neg_cnt[i]

  else:
    i+=1

print(top_pos_keys)

NameError: ignored

In [ ]:
worldcloud = WordCloud(stopwords=stopwords).generate(' '.join(top_pos_keys))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.savefig('wordcloud_pos.png', pad_inches=0.1)
plt.show()

worldcloud = WordCloud(stopwords=stopwords).generate(' '.join(top_neg_keys))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.savefig('wordcloud_neg.png', pad_inches=0.1)
plt.show()

import matplotlib.pyplot as plt
import seaborn as sns

X=['positive', 'negative']
freq=[pos_cnt, neg_cnt]

plt.bar(X, freq, color=['limegreen', 'red'])
plt.savefig('./count.png', dpi=200, bbox_inches='tight', pad_inches=0.1)
plt.show()

top_pos_keys=top_pos_keys[:20]
top_pos_cnt=top_pos_cnt[:20]

top_neg_keys=top_neg_keys[:20]
top_neg_cnt=top_neg_cnt[:20]

colors=sns.color_palette('YlGn', 20)
plt.barh(top_pos_keys[::-1], top_pos_cnt[::-1], color=colors)
plt.savefig('./positive.png', dpi=200, bbox_inches='tight', pad_inches=0.1)
plt.show()

colors=sns.color_palette('Reds', 20)
plt.barh(top_neg_keys[::-1], top_neg_cnt[::-1], color=colors)
plt.savefig('./negative.png', dpi=200, bbox_inches='tight', pad_inches=0.1)
plt.show()

In [20]:
# !pip install python-docx
from docx import Document
from docx.shared import Inches, Pt

document = Document()
document.add_heading('Sentiment Analysis Result', level = 0)
p = document.add_paragraph(style='ListNumber')
p.add_run().font.size=Pt(12)
p.add_run('Classification').bold=True
document.add_picture('count.png', width=Inches(5.5))
p = document.add_paragraph("The number of positive response: "+str(pos_cnt)+" ("+str(round(pos_cnt*100/(pos_cnt+neg_cnt),2))+"%)")
p = document.add_paragraph("The number of negative response: "+str(neg_cnt)+" ("+str(round(neg_cnt*100/(pos_cnt+neg_cnt),2))+"%)")
document.add_picture('wordcloud_total.png', width=Inches(5))

# Positive
p = document.add_paragraph(style='ListNumber')
p.add_run('Positive').bold=True
p = document.add_paragraph("The number of positive response: "+str(pos_cnt), style='ListBullet2')
p = document.add_paragraph("Top 20 frequent words: ", style='ListBullet2')
document.add_picture('positive.png', width=Inches(5.5))

table = document.add_table(rows=21, cols=2, style="Table Grid")
for i in range(21):
  table.cell(i,0).width=Inches(1.25)
  table.cell(i,1).width=Inches(1.25)

table.rows[0].cells[0].text = "word"
table.rows[0].cells[1].text = "count"

for i in range(20):
  table.rows[0].cells[0].text=top_pos_keys[i]
  table.rows[0].cells[1].text=str(top_pos_cnt[i])

document.add_picture('wordcloud_pos.png', width=Inches(5))
document.add_page_break()

#Negative
p = document.add_paragraph(style='ListNumber')
p.add_run('Negative').bold=True
p = document.add_paragraph("The number of negative response: "+str(neg_cnt), style='ListBullet2')
p = document.add_paragraph("Top 20 frequent words: ", style='ListBullet2')
document.add_picture('negative.png', width=Inches(5.5))

table = document.add_table(rows=21, cols=2, style="Table Grid")
for i in range(21):
  table.cell(i,0).width=Inches(1.25)
  table.cell(i,1).width=Inches(1.25)

table.rows[0].cells[0].text = "word"
table.rows[0].cells[1].text = "count"

for i in range(20):
  table.rows[0].cells[0].text=top_neg_keys[i]
  table.rows[0].cells[1].text=str(top_neg_cnt[i])

document.add_picture('wordcloud_neg.png', width=Inches(5))

document.save("result.docx")